In [7]:
import numpy as np
import pandas as pd
import math, time, random
from typing import Tuple
import gym 
import random


In [8]:
#iports of the Environment
from gym import spaces
from gym.utils import seeding
# from os import path


# Notes Section

Every environment comes with an action_space and an observation_space. These attributes are of type Space, and they describe the format of valid actions and observations:
The most common are Discrete and Box"Continous space".
##### So what will we do?
First I thought that we should have two action spaces: select_job {0: not_selected, 1:select} and update_job_state {0:not_released,1:released,2:starved}. 
Then I thought that we should have one action space select_job and based on it the state of each job shuld chaneg and hene the job_state should be in the observation_space.

### Passing the Parameters to the Workload-Generator

In [9]:
#Making them global to other notebooks on the same server
job_num=5
total_load=0.3
lo_per=0.2
job_density=5 #as suggested by the paper
%store job_num
%store total_load
%store lo_per
%store job_density


Stored 'job_num' (int)
Stored 'total_load' (float)
Stored 'lo_per' (float)
Stored 'job_density' (int)


### Uploading the workload/dataset 

In [10]:
workload_df= pd.read_excel(r'C:\Users\Youssef\Desktop\pythonjobgenerator\workload.xlsx')
workload_df.drop(workload_df.columns[workload_df.columns.str.contains('unnamed',case = False)],axis = 1, inplace = True)
#Now we have two copies of the workload, one as datafram and one as numpy array
empty_arr = [None] * job_num
empty_arr2 = [0] * job_num

workload_df.insert(4, "Job State", empty_arr, True) 
workload_df.insert(5, "Selected", empty_arr2, True) 
workload=np.array(workload_df)
temp_df=workload_df
workload_df


,Releases,Deadlines,Processing,Criticality,Job State,Selected
0,1.1560,3.1475,0.3809,1,None,0
1,0.4358,3.1553,0.0883,1,None,0
2,0.3846,3.2697,0.4495,1,None,0
3,0.0000,7.0160,0.7605,1,None,0
4,0.4672,9.7825,1.2134,1,None,0


### Constructor/Initialization

In [11]:
class MC_agent(gym.Env):
    def __init__(self,x,y,z,g,j):
        self.time=j
#         self.job_num=x
#         self.total_load=y
#         self.lo_per=z
#         self.job_density=g #as suggested by the paper
#         store_params()
# #              Action:
# #              0: not selected job 
# #              1: selected
        self.action_space_select_job = spaces.Discrete(2) 
        self.viewer=None
#         self.observation_space = spaces.Box() 

    def step(self):
        action=0
        finished=False
        
        while(finished==False):
            for el in workload:
                if(el[:][0]>self.time):
                    el[:][4]=0 #not_released yet
                elif(el[:][0]<=self.time):
                    el[:][4]=1 #released
                if(el[:][1]<=self.time):
                    el[:][4]=2 #starved
            self.time=self.time+1

            for el in workload:
                action=self.action_space_select_job.sample()
                if ((action==1)and(el[:][5]==0) and (el[:][4]==1)): #I can only select a job if the action is 1 and it hasnot been seleted before and it is released
                    el[:][5]=1
                    self.time=self.time+ el[:][2] #time=time+processing
                    self.time=self.time-1 #As it should be updated only by processing
                    break
                
            workload_df = pd.DataFrame({'Releases': workload[:, 0], 'Deadlines': workload[:, 1],'Processing': workload[:, 2], 'Criticality': workload[:, 3],"Job State":workload[:,4],"Selected":workload[:,5] })
            print("Workload at time: ", self.time-1)
            print( workload_df)
            if(self.time>=workload[job_num-1][1]):
                workload[job_num-1][4]=2
                finished=True
    def reset(self):
        workload_df=temp_df
#         self.time=0 #This can be removed as in the initialization it is initialized
    
    

In [12]:
#Main
agent= MC_agent(25,0.3,0.5,5,0)
agent.reset()
sel_job_act=agent.action_space_select_job.sample()
agent.step()
# agent.time( 0.3527)
# agent.step()


Workload at time:  -0.23950000000000005
  Releases Deadlines Processing Criticality Job State Selected
0    1.156    3.1475     0.3809           1         0        0
1   0.4358    3.1553     0.0883           1         0        0
2   0.3846    3.2697     0.4495           1         0        0
3        0     7.016     0.7605           1         1        1
4   0.4672    9.7825     1.2134           1         0        0
Workload at time:  0.20999999999999996
  Releases Deadlines Processing Criticality Job State Selected
0    1.156    3.1475     0.3809           1         0        0
1   0.4358    3.1553     0.0883           1         1        0
2   0.3846    3.2697     0.4495           1         1        1
3        0     7.016     0.7605           1         1        1
4   0.4672    9.7825     1.2134           1         1        0
Workload at time:  0.5909
  Releases Deadlines Processing Criticality Job State Selected
0    1.156    3.1475     0.3809           1         1        1
1   0.4358   

#### Tasks couldn't finish
passing parameters better,
rendering,
observation space defintion 